<a href="https://colab.research.google.com/github/ich-youness/Detect-LLM-Generated-ArabicText/blob/main/Detect_LLM_Generated_Arabic_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

detect_llm_generated_arabic_text_path = kagglehub.competition_download('detect-llm-generated-arabic-text')
kaggle_arabert_tensorflow2_arabert_1_path = kagglehub.model_download('kaggle/arabert/TensorFlow2/arabert/1')
ynsichb_aubmindlabbert_base_arabertv2_transformers_default_1_path = kagglehub.model_download('ynsichb/aubmindlabbert-base-arabertv2/Transformers/default/1')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pyarabic

In [ ]:
import matplotlib.pyplot as plt
import pyarabic.araby as araby
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
import re

In [ ]:
train_data = pd.read_csv("/kaggle/input/detect-llm-generated-arabic-text/train_set.csv")
test_data = pd.read_csv("/kaggle/input/detect-llm-generated-arabic-text/test_set.csv")


In [ ]:
train_data.head(10)

In [ ]:
train_data["generated"].value_counts().plot(kind="bar")

In [ ]:
arabic_stopwords = stopwords.words("arabic")
print(arabic_stopwords)

# Preprocessing

In [ ]:
# for  word in arabic_stopwords:
#     if any(word in text for text in train_data["text"]):
        # print(f"the word deleted: {word}")




**Remove Stopwords**

In [ ]:
def process(text):
    text = araby.strip_tashkeel(text)
    text = araby.normalize_hamza(text)

    words = text.split()
    filtered_text = [word for word in words if word not in arabic_stopwords]
    return " ".join(filtered_text)


In [ ]:
train_data["filtered_text"] = train_data["text"].apply(process)

In [ ]:
train_data.head()

**Stemmer**

In [ ]:
def stemmer(text):
    stemmer = ISRIStemmer()

    text = stemmer.stem(text)
    text = stemmer.pre32(text)
    text = stemmer.suf32(text)

    return text



In [ ]:
train_data["filtered_text"] = train_data["filtered_text"].apply(stemmer)


In [ ]:
train_data.head()

In [ ]:
def normalizeArabic(text):
    text = text.strip()
    ## replace re.sub(<pattern>, <replacement>, string, <count>, <flags>)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)

    #remove repetetions
    text = re.sub("[إأٱآا]", "ا", text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('ييي', 'ي')
    text = text.replace('اا', 'ا')

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)

    # Remove specified special characters
    text = re.sub(r"[<<\n،»»;',-]", "", text)

    # Remove longation
    text = re.sub(r'(.)\1+', r"\1\1", text)

    #Strip vowels from a text, include Shadda.
    text = araby.strip_tashkeel(text)

    #Strip diacritics from a text, include harakats and small lettres The striped marks are
    text = araby.strip_diacritics(text)
    text=''.join([i for i in text if not i.isdigit()])
    return text


In [ ]:
train_data["filtered_text"] = train_data["filtered_text"].apply(normalizeArabic)

In [ ]:
train_data.head()

In [ ]:
def Removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

In [ ]:
train_data["filtered_text"] = train_data["filtered_text"].apply(Removing_punctuations)

In [ ]:
train_data.head()

# Trainning

In [ ]:
!pip install transformers

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="aubmindlab/bert-base-arabertv2")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv2")

In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model for sequence classification
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification


In [ ]:
from sklearn.model_selection import train_test_split

X,Y = train_data["filtered_text"], train_data["generated"]

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=32)

In [ ]:
train_data.head()

In [ ]:
def encode_text(texts, tokenizer, max_length=256):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='tf'
    )

# Tokenize the train and test data
train_encodings = encode_text(x_train.tolist(), tokenizer)
test_encodings = encode_text(x_test.tolist(), tokenizer)

# # Prepare the datasets as TensorFlow datasets
# import tensorflow as tf
# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings),
#     train_data["text_id"].values  # Replace with the actual label column
# ))

# test_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(test_encodings),
#     test_data["text_id"].values  # Replace with the actual label column
# ))

# train_dataset = train_dataset.batch(16)
# test_dataset = test_dataset.batch(16)

In [ ]:
# def tokenize_data(texts):
#     return tokenizer(
#         list(texts),
#         max_length=128,
#         truncation=True,
#         padding="max_length",
#         return_tensors="tf",
#     )
# x_train_tokenized = tokenize_data(x_train)
# x_test_tokenized = tokenize_data(x_test)


In [ ]:
# Tokenize the training and testing data
train_encodings = tokenizer(x_train.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(x_test.tolist(), truncation=True, padding=True, max_length=128)

In [ ]:
import tensorflow as tf

# Convert the tokenized data into TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
BATCH_SIZE = 16
train_dataset = train_dataset.shuffle(len(x_train)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)


loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [ ]:
import tensorflow as tf

# Check if TensorFlow detects the GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Print GPU details (optional)
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    print(device)

In [ ]:
EPOCHS = 6
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS
)


In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy:.2f}")


In [ ]:
model.save("arabert_classification_model")
